In [27]:
import subprocess as proc

types = {'INPUT': 'unsigned char',
         'COEFFICIENTS': 'unsigned char',
         'INDEX': 'unsigned int',
         'RESULTS': 'unsigned int',
         'BAKC_PROJECTED': 'unsigned int',
         'NORMALIZED': 'unsigned int'}

def generateCythonCode(verbose = False):
    output=[]

    with open('cython_template.txt', 'r') as f:
        i = 1
        for line in f:
            code = line
            if 'cpdef' in code:
                if (code.count(',')+1) != code.count('{'):
                    print(code)
                    print("Warning: Unwrapped type detected at line %i"%i)
                    print('#'*100)
            for typ in types:
                indicator = '{%s}'%typ
                if indicator in code:
                    code = code.replace(indicator,types[typ])
            output.append(code)
            i+=1

    with open('functions.pyx', 'w') as f:
        f.write(''.join(output))
        
    if verbose:
        print(''.join(output))

def compileCythonCode():
    print("#"*100)
    print("Compiling ...")
    print("#"*100)
    p1 = proc.run('python setupFinal.py build_ext --inplace', shell=True, capture_output=True)
    print(p1.stdout.decode())
    if 'Finished generating code' in p1.stdout.decode():
        print("#"*100)
        print("Generating report ...")
        print("#"*100)
        p2 = proc.run('cython -a functions.pyx', shell=True, capture_output=True)
        print("Compilation finished!")
        
generateCythonCode(True)
compileCythonCode()

cimport cython

@cython.wraparound(False)
@cython.boundscheck(False)            
cpdef zeros_int32(unsigned int[::1] arr):
    cdef unsigned int x
    for x in range(arr.shape[0]):
        arr[x] = 0

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.cdivision(True)
cpdef normalize(unsigned int[::1] BP_flat, unsigned int[::1] norm_flat):
    cdef unsigned int x
    for x in range(BP_flat.shape[0]):
        BP_flat[x] = BP_flat[x]//norm_flat[x]

@cython.wraparound(False)
@cython.boundscheck(False)            
cpdef sample(unsigned char[::1] img_flat, unsigned char[::1] coeffs, unsigned int[::1] idx, unsigned int[::1] result_flat):
    cdef unsigned int x
    for x in range(img_flat.shape[0]):
        if coeffs[x] > 0:
            result_flat[idx[x]] += img_flat[x]*coeffs[x]

@cython.wraparound(False)
@cython.boundscheck(False)            
cpdef sampleRGB(unsigned char[::1] R, unsigned char[::1] G, unsigned char[::1] B, unsigned char[::1] coeffs, unsigned int[::1] idx, unsigne